In [1]:
%matplotlib inline
from pycocotools.coco import COCO
import numpy as np
import skimage.io as io
import imageio
import matplotlib.pyplot as plt
import pylab
from tqdm import tqdm_notebook
import imageio.core.util
import os

# for multiprocessing
from multiprocessing import Pool

pylab.rcParams['figure.figsize'] = (8.0, 10.0)

# 1. Setting Hyper-Params

In [2]:
# Root Dir
COCO_DIR = './COCO'

In [3]:
# Train data dir
TRAIN = 'train2017'
TRAIN_ANN_FILE = COCO_DIR + '/annotations/instances_' + TRAIN + '.json'
TRAIN_OUT_DIR = COCO_DIR + '/annotations/sementic_' + TRAIN + '/'

In [4]:
# Val data dir
VAL = 'val2017'
VAL_ANN_FILE = COCO_DIR + '/annotations/instances_' + VAL + '.json'
VAL_OUT_DIR = COCO_DIR + '/annotations/sementic_' + VAL + '/'

In [5]:
# Warning ignore
def ignore_warnings(*args, **kwargs):
    pass

imageio.core.util._precision_warn = ignore_warnings

# 2.Train mask generate

In [6]:
# initialize COCO api for instance annotations and caption annocations
coco = COCO(TRAIN_ANN_FILE)

loading annotations into memory...
Done (t=10.70s)
creating index...
index created!


In [7]:
# display COCO categories and supercategories
cats = coco.loadCats(coco.getCatIds())
nms=[cat['name'] for cat in cats]
print('COCO categories: \n{}\n'.format(' '.join(nms)))

nms = set([cat['supercategory'] for cat in cats])
print('COCO supercategories: \n{}'.format(' '.join(nms)))

COCO categories: 
person bicycle car motorcycle airplane bus train truck boat traffic light fire hydrant stop sign parking meter bench bird cat dog horse sheep cow elephant bear zebra giraffe backpack umbrella handbag tie suitcase frisbee skis snowboard sports ball kite baseball bat baseball glove skateboard surfboard tennis racket bottle wine glass cup fork knife spoon bowl banana apple sandwich orange broccoli carrot hot dog pizza donut cake chair couch potted plant bed dining table toilet tv laptop mouse remote keyboard cell phone microwave oven toaster sink refrigerator book clock vase scissors teddy bear hair drier toothbrush

COCO supercategories: 
sports indoor kitchen vehicle person electronic outdoor appliance furniture animal accessory food


In [8]:
def trainAnns(idx):
    ids = coco.getImgIds()[idx]
    catids = coco.getCatIds()
    imgIds = coco.getImgIds(imgIds = ids)
    img = coco.loadImgs(imgIds[np.random.randint(0,len(imgIds))])[0]
    I = io.imread(COCO_DIR + '/' + TRAIN + '/' +img['file_name']).astype(np.uint8)
    
    anns_ids = coco.getAnnIds(imgIds=img['id'], catIds=catids, iscrowd=None)
    anns = coco.loadAnns(anns_ids)
    anns_img = np.zeros((img['height'],img['width']))
    
    for i in range(len(anns)):
        anns_img = coco.annToMask(anns[i])
        
    #io.imshow(anns_img, cmap='gray');
    #io.imsave(TRAIN_OUT_DIR+'{0:0>12}.png'.format(img['id']), anns_img, check_contrast=False)
    imageio.imwrite(TRAIN_OUT_DIR+'{0:0>12}.png'.format(img['id']), anns_img)

In [9]:
# RUN on imageio==2.8.0
with Pool(processes=16) as p:
    max_ = len(coco.getImgIds())
    with tqdm_notebook(total=max_) as pbar:
        for i, _ in tqdm_notebook(enumerate(p.imap_unordered(trainAnns, range(max_)))):
            pbar.update()

# 3.Validation mask generate

In [10]:
# initialize COCO api for instance annotations and caption annocations
coco = COCO(VAL_ANN_FILE)

loading annotations into memory...
Done (t=0.34s)
creating index...
index created!


In [11]:
# display COCO categories and supercategories
cats = coco.loadCats(coco.getCatIds())
nms=[cat['name'] for cat in cats]
print('COCO categories: \n{}\n'.format(' '.join(nms)))

nms = set([cat['supercategory'] for cat in cats])
print('COCO supercategories: \n{}'.format(' '.join(nms)))

COCO categories: 
person bicycle car motorcycle airplane bus train truck boat traffic light fire hydrant stop sign parking meter bench bird cat dog horse sheep cow elephant bear zebra giraffe backpack umbrella handbag tie suitcase frisbee skis snowboard sports ball kite baseball bat baseball glove skateboard surfboard tennis racket bottle wine glass cup fork knife spoon bowl banana apple sandwich orange broccoli carrot hot dog pizza donut cake chair couch potted plant bed dining table toilet tv laptop mouse remote keyboard cell phone microwave oven toaster sink refrigerator book clock vase scissors teddy bear hair drier toothbrush

COCO supercategories: 
sports indoor kitchen vehicle person electronic outdoor appliance furniture animal accessory food


In [12]:
def valAnns(idx):
    ids = coco.getImgIds()[idx]
    catids = coco.getCatIds()
    imgIds = coco.getImgIds(imgIds = ids)
    img = coco.loadImgs(imgIds[np.random.randint(0,len(imgIds))])[0]
    I = io.imread(COCO_DIR + '/' + VAL + '/' +img['file_name']).astype(np.uint8)

    anns_ids = coco.getAnnIds(imgIds=img['id'], catIds=catids, iscrowd=None)
    anns = coco.loadAnns(anns_ids)
    anns_img = np.zeros((img['height'],img['width']))
    
    for i in range(len(anns)):
        anns_img = coco.annToMask(anns[i])
        
    #io.imshow(anns_img, cmap='gray');
    #io.imsave(VAL_OUT_DIR+'{0:0>12}.png'.format(img['id']), anns_img, check_contrast=False)
    imageio.imwrite(VAL_OUT_DIR+'{0:0>12}.png'.format(img['id']), anns_img)

In [13]:
# RUN on imageio==2.8.0
with Pool(processes=16) as p:
    max_ = len(coco.getImgIds())
    with tqdm_notebook(total=max_) as pbar:
        for i, _ in tqdm_notebook(enumerate(p.imap_unordered(valAnns, range(max_)))):
            pbar.update()

In [14]:
#from discord_webhook import DiscordWebhook
#url = 'https://discordapp.com/api/webhooks/710208007618822145/4yUFIEoTa7kZFOhyJpSkalNn2NysrM6p5PFVG5iBDkt1ikJxBPwV3_J4FDYi40THgxvl'
#webhook = DiscordWebhook(url=url, content='segment generation completed...')
#response = webhook.execute()